<a href="https://colab.research.google.com/github/kumarchandan/END/blob/main/week9/1_Seq2Seq_Customer_support_on_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Customer Support on Twitter:

https://www.kaggle.com/thoughtvector/customer-support-on-twitter

This Kaggle dataset includes more than 3 million tweets and responses from leading brands on Twitter.

#### 2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torchtext
from torchtext.data import Field, BucketIterator
from sklearn.model_selection import train_test_split

import spacy
import numpy as np

import random
import math
import time

Then set a random seed for deterministic results/reproducability.

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!cp '/content/drive/MyDrive/1 Projects/END/Week9/tweets_min.csv' .

In [7]:
tweets = pd.read_csv('tweets_min.csv')
tweets.shape

(100000, 2)

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

Preprocess data

In [8]:
TWEET = torchtext.data.Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('tweet', TWEET), ('response', TWEET)]

In [9]:
tweets.head()

,tweet,response
0,@sprintcare is the worst customer service,@115712 Can you please send us a private messa...
1,@115714 y’all lie about your “great” connectio...,@115713 H there! We'd definitely like to work ...
2,"@115714 whenever I contact customer support, t...",@115715 Please send me a private message so th...
3,actually that's a broken link you sent me and ...,@115716 The information pertaining to the acco...
4,"Yo @Ask_Spectrum, your customer service reps a...","@115717 Hello, My apologies for any frustratio..."


In [10]:
dataset = torchtext.data.TabularDataset(
    path='tweets_min.csv',
    format='CSV',
    fields=fields,
    skip_header=True
  )
# Train/val split
# (train, valid) = train_test_split(df, test_size=0.15)
(train, valid) = dataset.split(split_ratio=[0.85, 0.15])
print(len(train), len(valid))

85000 15000


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [11]:
MAX_VOCAB_SIZE = 10_000

TWEET.build_vocab(train, max_size=MAX_VOCAB_SIZE)

print(f'Size of vocab: {len(TWEET.vocab)}')

Size of vocab: 10004


Finally, define the `device` and create our iterators.

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
BATCH_SIZE = 128

train_iterator, valid_iterator = torchtext.data.BucketIterator.splits(
    (train, valid),
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.tweet),
    device = device
)

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

In [15]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [17]:
INPUT_DIM = len(TWEET.vocab)
OUTPUT_DIM = len(TWEET.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [18]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(128, 256)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(384, 256)
    (fc_out): Linear(in_features=640, out_features=10004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

We print out the number of parameters.

Even though we only have a single layer RNN for our encoder and decoder we actually have **more** parameters  than the last model. This is due to the increased size of the inputs to the GRU and the linear layer. However, it is not a significant amount of parameters and causes a minimal amount of increase in training time (~3 seconds per epoch extra).

In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,763,092 trainable parameters


We initiaize our optimizer.

In [20]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [21]:
TRG_PAD_IDX = TWEET.vocab.stoi[TWEET.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [22]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.tweet
        trg = batch.response
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [23]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.tweet
            trg = batch.response

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [25]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 59s
	Train Loss: 5.031 | Train PPL: 153.153
	 Val. Loss: 5.855 |  Val. PPL: 349.149
Epoch: 02 | Time: 5m 0s
	Train Loss: 4.356 | Train PPL:  77.961
	 Val. Loss: 5.705 |  Val. PPL: 300.452
Epoch: 03 | Time: 5m 0s
	Train Loss: 3.945 | Train PPL:  51.655
	 Val. Loss: 5.654 |  Val. PPL: 285.519
Epoch: 04 | Time: 4m 58s
	Train Loss: 3.711 | Train PPL:  40.899
	 Val. Loss: 5.662 |  Val. PPL: 287.867
Epoch: 05 | Time: 4m 59s
	Train Loss: 3.592 | Train PPL:  36.290
	 Val. Loss: 5.674 |  Val. PPL: 291.086
Epoch: 06 | Time: 4m 59s
	Train Loss: 3.504 | Train PPL:  33.235
	 Val. Loss: 5.592 |  Val. PPL: 268.344
Epoch: 07 | Time: 4m 59s
	Train Loss: 3.451 | Train PPL:  31.532
	 Val. Loss: 5.678 |  Val. PPL: 292.322
Epoch: 08 | Time: 4m 59s
	Train Loss: 3.401 | Train PPL:  30.000
	 Val. Loss: 5.686 |  Val. PPL: 294.833
Epoch: 09 | Time: 4m 59s
	Train Loss: 3.354 | Train PPL:  28.608
	 Val. Loss: 5.674 |  Val. PPL: 291.096
Epoch: 10 | Time: 4m 59s
	Train Loss: 3.311 | Train PPL: 